What i want to do: 
- read in postcode dataset
- loop through the code point files 
- join postcodes in each file 
- make sure these are saved and each time loop through a file it doesn't delete earlier merges. 

In [1]:
# Set libraries
import pandas as pd
import glob
import os
import numpy as np
import datetime

In [2]:
# Read in postcode dataset
UU_postcode = pd.read_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Drinking_water\United Utilities\UU Zonal nitrate for imperial final 2000-2010_ed.csv")

In [ ]:
UU_postcode

- Get rid of the white space in the postcodes that have XXXX XXX to XXXXXXX characters so that they match with the OS codepoint dataset

In [4]:
UU_postcode['Postcode'] = UU_postcode['Postcode'].str.strip()

UU_postcode.to_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Drinking_water\United Utilities\UU_test.csv")

In [5]:
mask = (UU_postcode['Postcode'].str.len()>7)
mask2 = (UU_postcode['Postcode'].str.len()<8)

In [6]:
df1 = UU_postcode.loc[mask]

In [7]:
df2 = UU_postcode.loc[mask2]

In [ ]:
df1

In [ ]:
df2

In [10]:
df1['First']= df1['Postcode'].str[:4]

C:\Users\ansheeha\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
df1['Last']= df1['Postcode'].str[5:8]

C:\Users\ansheeha\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
df1['New_pstcd'] = df1['First'].map(str) + df1['Last']

C:\Users\ansheeha\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df1

df1.to_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Drinking_water\United Utilities\UU_df1.csv")

In [14]:
df1_n = df1[['Date', 'New_pstcd', 'Nitrate', 'Qualifier', 'Zone desc']]

In [15]:
df1_n = df1_n.rename(columns={'New_pstcd':'Postcode'})

In [16]:
df_all = pd.concat([df1_n, df2])

In [17]:
df_all = df_all.set_index('Postcode')

In [ ]:
df_all

- read in the header file for the Code point postcode files 
- set the column headings to the first row (which contains the header names)
- set the index to the column which says postcode
- delete the row with the duplicate of column headings- so end up with a table of just the column headings

In [19]:
labels2 = pd.read_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Doc\Code-Point_Open_Column_Headers.csv", header = None)

In [20]:
labels2.columns = labels2.iloc[0]

In [21]:
labels2

,Postcode,Positional_quality_indicator,Eastings,Northings,Country_code,NHS_regional_HA_code,NHS_HA_code,Admin_county_code,Admin_district_code,Admin_ward_code
0,Postcode,Positional_quality_indicator,Eastings,Northings,Country_code,NHS_regional_HA_code,NHS_HA_code,Admin_county_code,Admin_district_code,Admin_ward_code


In [22]:
labels2 = labels2.set_index('Postcode')

In [23]:
labels2 = labels2.reindex(labels2.index.drop('Postcode'))

- merge the SWW postcodes and the labels to create a dataframe in which we can fill with the OS easting and northing information

In [24]:
df_all = df_all.merge(labels2, how='left', left_index=True, right_index=True)

In [ ]:
df_all

- read in the labels data again so that the OS codepoint files can be merged with the column headings

In [4]:
labels = pd.read_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Doc\Code-Point_Open_Column_Headers.csv", header = None)

- Create a list of the files within the codepoint folder (containing all the different postcodes) to use within the for loop

In [5]:
files = glob.glob(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\*")

In [6]:
files

['S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\tw.csv',
 'S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\dy.csv',
 'S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\la.csv',
 'S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\hd.csv',
 'S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\sl.csv',
 'S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\dg.csv',
 'S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\ph.csv',
 'S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\ws.csv',
 'S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\e.csv',
 'S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\gu.csv',
 'S:\\Projects\\SAHSU_Share\\Diabetes\\Datasets\\Postcodes\\codepo_gb\\Data\\CSV\\ig.csv',


- create a loop to insert data into the df_all dataset
    - read the postcode file in
    - merge the labels and postcode file so the postcode file has column headings 
    - set the column headings as actual column headings 
    - set the index to postcode
    - delete the duplicate postcode row
    - insert a column 'date' because the table we are filling has a date column
    - ensure that the date column is in datetime format
    - insert the postcode information into the SWW postcode dataset
    - delete all of the extra postcodes that aren't in the SWW postcode dataset
    - display that the postcode file has been processed 

In [30]:
for file in files:
    postcode = pd.read_csv(r'{fname}'.format(fname=file), header=None)
    data = pd.concat([labels,postcode])
    data.columns = data.iloc[0]
    data = data.set_index(['Postcode'])
    data = data.reindex(data.index.drop('Postcode'))
    data.insert(0, 'Date', 'NaN')
    data.insert(1,'Nitrate', 'NaN')
    data.insert(2,'Qualifier', 'NaN')
    data.insert(3,'Zone desc', 'NaN')
    data['Date']= pd.to_datetime(data['Date'])
    df_all = df_all.combine_first(data)
    df_all = df_all[pd.notnull(df_all['Date'])]
    
    
    print('Processed {fname}'.format(fname=file))

Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\tw.csv
Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\dy.csv
Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\la.csv
Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\hd.csv
Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\sl.csv
Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\dg.csv
Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\ph.csv
Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\ws.csv
Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\e.csv
Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\gu.csv
Processed S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\ig.csv
Processed S:\Projects\SAHSU_Share

C:\Users\ansheeha\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

- save the dataset

In [31]:
df_all.to_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Drinking_water\United Utilities\UU_Northing_Easting.csv")

- rerunning the code on the dataset using a smaller number of the OS Codepoint files 

In [ ]:
# Read in postcode dataset
UU_postcode = pd.read_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Drinking_water\United Utilities\UU_Unmatched.csv")

In [ ]:
UU_postcode

In [ ]:
UU_postcode['Postcode'] = UU_postcode['Postcode'].str.strip()

In [ ]:
df_all = UU_postcode

In [ ]:
df_all = df_all.set_index('Postcode')

In [ ]:
df_all

In [ ]:
labels2 = pd.read_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Doc\Code-Point_Open_Column_Headers.csv", header = None)

In [ ]:
labels2.columns = labels2.iloc[0]

In [ ]:
labels2

In [ ]:
labels2 = labels2.set_index('Postcode')

In [ ]:
labels2 = labels2.reindex(labels2.index.drop('Postcode'))

In [ ]:
df_all = df_all.merge(labels2, how='left', left_index=True, right_index=True)

In [ ]:
df_all

In [ ]:
labels = pd.read_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Doc\Code-Point_Open_Column_Headers.csv", header = None)

In [ ]:
files = glob.glob(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Drinking_water\United Utilities\Postcodes\*")

In [ ]:
files

In [ ]:
for file in files:
    postcode = pd.read_csv(r'{fname}'.format(fname=file), header=None)
    data = pd.concat([labels,postcode])
    data.columns = data.iloc[0]
    data = data.set_index(['Postcode'])
    data = data.reindex(data.index.drop('Postcode'))
    data.insert(0, 'Date', 'NaN')
    data.insert(1,'Nitrate', 'NaN')
    data.insert(2,'Qualifier', 'NaN')
    data.insert(3,'Zone desc', 'NaN')
    data['Date']= pd.to_datetime(data['Date'])
    df_all = df_all.combine_first(data)
    df_all = df_all[pd.notnull(df_all['Date'])]
    
    
    print('Processed {fname}'.format(fname=file))

In [ ]:
df_all.to_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Drinking_water\United Utilities\UU_unmatchedmatched.csv")

## cutting down unmatched postcodes to postcode district level and matching to postcode district centroid file

In [84]:
unmatched = pd.read_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Drinking_water\United Utilities\UU_unmatched_2.csv")

In [ ]:
unmatched

In [86]:
mask1 = (unmatched['Postcode'].str.len()>6)
mask2 = (unmatched['Postcode'].str.len()==6)
mask3 = (unmatched['Postcode'].str.len()==5)
mask4 = (unmatched['Postcode'].str.len()<5)


In [87]:
df1 = unmatched.loc[mask1]
df2 = unmatched.loc[mask2]
df3 = unmatched.loc[mask3]
df4 = unmatched.loc[mask4]

In [ ]:
 df3

In [89]:
df2['First']=df2['Postcode'].str[:3]

C:\Users\ansheeha\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [90]:
df2['Last']= df2['Postcode'].str[3:]

C:\Users\ansheeha\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [91]:
df2['New_pstcd'] = df2['First'].map(str) + " " + df2['Last']

C:\Users\ansheeha\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [92]:
df2_n = df2[['Date', 'New_pstcd', 'Nitrate', 'Qualifier', 'Zone desc']]

In [93]:
df2_n = df2_n.rename(columns={'New_pstcd':'Postcode'})

In [94]:
df_all = pd.concat([df2_n, df1, df4])

In [ ]:
df_all

In [96]:
unmatched = df_all

In [ ]:
unmatched

In [98]:
unmatched['District']= unmatched['Postcode'].str[:4]

In [99]:
unmatched['District'] = unmatched['District'].str.strip()

- read in unmatched SWW data 
- match with the postcode district centroid files
- match with postcode csv

In [100]:
unmatched = unmatched.set_index(['District'])

In [102]:
postcode_dist = pd.read_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\Post_d_centroids.csv")

In [ ]:
postcode_dist

In [104]:
postcode_dist = postcode_dist.rename(columns={'X_Coord':'Eastings', 'Y_Coord':'Northings'})

In [105]:
postcode_dist = postcode_dist.set_index(['District'])

In [106]:
postcode_dist = postcode_dist[['Eastings', 'Northings']]

In [107]:
postcode_dist.insert(0, 'Postcode', 'NaN')

In [108]:
postcode_dist.insert(1, 'Date', 'NaN')
postcode_dist.insert(2, 'Nitrate', 'NaN')
postcode_dist.insert(3, 'Qualifier', 'NaN')
postcode_dist.insert(4, 'Zone desc', 'NaN')

In [109]:
postcode_dist['Date']= pd.to_datetime(postcode_dist['Date'])

In [ ]:
postcode_dist

In [111]:
unmatched = unmatched.combine_first(postcode_dist)
unmatched = unmatched[pd.notnull(unmatched['Date'])]

In [ ]:
unmatched

In [113]:
unmatched.to_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Drinking_water\United Utilities\UU_unmatched_2_matched.csv")

In [33]:
postcode = pd.read_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\all_postcodes_cut.csv")

In [34]:
postcode

,Postcode,Eastings,Northings
0,AB101AB,394235,806529
1,AB101AF,394181,806429
2,AB101AG,394230,806469
3,AB101AH,394371,806359
4,AB101AL,394326,806522
5,AB101AN,394367,806541
6,AB101AP,394309,806459
7,AB101AQ,394371,806359
8,AB101AR,394371,806359
9,AB101AS,394198,806385


In [ ]:
postcode = postcode.rename(columns={'Postcode':'Postcode_editted'})

In [ ]:
postcode = postcode.set_index(['Postcode_editted'])

In [ ]:
postcode.insert(0, 'Postcode', 'NaN')

In [ ]:
postcode.insert(2, 'Nitrate', 'NaN')
postcode.insert(4, 'date', 'NaN')

In [ ]:
postcode['date']= pd.to_datetime(postcode['date'])

In [ ]:
unmatched = unmatched.combine_first(postcode)
unmatched = unmatched[pd.notnull(unmatched['date'])]

In [ ]:
unmatched

In [ ]:
unmatched.to_csv(r'S:\Projects\SAHSU_Share\Diabetes\Datasets\Drinking_water\SouthWest\editted\all_SWW_unmatched_matched.csv')